# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e compramos e vendemos commodities:
- Soja, Milho, Trigo, Petróleo, etc.

Precisamos pegar na internet, de forma automática, a cotação de todas as commodites e ver se ela está abaixo do nosso preço ideal de compra. Se tiver, precisamos marcar como uma ação de compra para a equipe de operações.

In [2]:
from selenium import webdriver

navegador = webdriver.Chrome()
navegador.get("https://www.google.com/")

In [3]:
import pandas as pd

tabela = pd.read_excel("commodities.xlsx")
display(tabela)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN
5,Açúcar,136.23,NaN,NaN
6,Café,1092.87,NaN,NaN
7,Ouro,321.77,NaN,NaN
8,Tilápia,9.05,NaN,NaN


In [4]:
# import unicodedata
# produto = unicodedata.normalize("NFKD", produto).encode("ascii", "ignore") # substituir tudo o que for caracter especial por caracter normal

for linha in tabela.index:
    produto = tabela.loc[linha, "Produto"] # procuro cada linha da coluna produto
    print(produto)
    
    produto = produto.replace("ó", "o").replace("ã", "a").replace("á", "a").replace(
    "ç", "c").replace("ú", "u").replace("é", "e")
    
    link = f"https://www.melhorcambio.com/{produto}-hoje"
    print(link)
    navegador.get(link)

    preco = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value') # xpath do valor da commodite, value é o atributo do código html
    preco = preco.replace(".", "").replace(",", ".")
    print(preco)
    tabela.loc[linha, "Preço Atual"] = float(preco) # adiciono o preço atual em cada linha da coluna preço atual
    
display(tabela)

Milho
https://www.melhorcambio.com/Milho-hoje
52.84
Soja
https://www.melhorcambio.com/Soja-hoje
142.50
Boi
https://www.melhorcambio.com/Boi-hoje
230.10
Petróleo
https://www.melhorcambio.com/Petroleo-hoje
426.01
Algodão
https://www.melhorcambio.com/Algodao-hoje
404.61
Açúcar
https://www.melhorcambio.com/Acucar-hoje
136.63
Café
https://www.melhorcambio.com/Cafe-hoje
835.83
Ouro
https://www.melhorcambio.com/Ouro-hoje
309.39
Tilápia
https://www.melhorcambio.com/Tilapia-hoje
9.53
Acabou


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,52.84,NaN
1,Soja,163.59,142.50,NaN
2,Boi,282.20,230.10,NaN
3,Petróleo,424.37,426.01,NaN
4,Algodão,497.76,404.61,NaN
5,Açúcar,136.23,136.63,NaN
6,Café,1092.87,835.83,NaN
7,Ouro,321.77,309.39,NaN
8,Tilápia,9.05,9.53,NaN


In [5]:
# preencher a coluna de comprar
tabela["Comprar"] = tabela["Preço Atual"] < tabela["Preço Ideal"]
display(tabela)

# exportar a base para o excel
tabela.to_excel("commodities_atualizado.xlsx", index=False)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,52.84,True
1,Soja,163.59,142.50,True
2,Boi,282.20,230.10,True
3,Petróleo,424.37,426.01,False
4,Algodão,497.76,404.61,True
5,Açúcar,136.23,136.63,False
6,Café,1092.87,835.83,True
7,Ouro,321.77,309.39,True
8,Tilápia,9.05,9.53,False


In [6]:
navegador.quit()